In [388]:
import pandas as pd
import numpy as np
import matplotlib as mpl
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
#%matplotlib notebook
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [389]:
df = pd.read_csv("properatti.csv")

### Exploración inicial

In [390]:
df.head(3)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.000,"-34.6618237,-58.5088387",-34.662,...,40.000,1127.273,1550.000,nan,nan,nan,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.000,"-34.9038831,-57.9643295",-34.904,...,nan,nan,nan,nan,nan,nan,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.000,"-34.6522615,-58.5229825",-34.652,...,55.000,1309.091,1309.091,nan,nan,nan,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...


In [391]:
df.shape

(121220, 26)

In [392]:
df.describe()

,Unnamed: 0,geonames_id,lat,lon,price,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses
count,121220.000,102503.000,69670.000,69670.000,100810.000,100810.000,100810.000,81892.000,101313.000,68617.000,87658.000,7899.000,47390.000,14262.000
mean,60609.500,3574442.318,-34.626,-59.266,468525.925,4229396.829,239700.577,233.795,133.050,2160.087,6912.216,17.452,3.081,5009.234
std,34993.344,354130.624,1.981,2.300,2260100.589,6904714.126,391323.876,1782.222,724.351,2759.289,28378.645,120.244,1.861,120440.256
min,0.000,3427208.000,-54.824,-75.679,0.000,0.000,0.000,0.000,0.000,0.600,1.510,1.000,1.000,1.000
25%,30304.750,3430234.000,-34.669,-58.727,110000.000,1583309.440,89733.880,50.000,45.000,1218.182,1550.000,1.000,2.000,1000.000
50%,60609.500,3433910.000,-34.598,-58.480,185000.000,2558452.500,145000.000,84.000,75.000,1800.000,2213.115,3.000,3.000,2000.000
75%,90914.250,3836668.000,-34.441,-58.396,420000.000,4675792.500,265000.000,200.000,150.000,2486.412,3355.549,6.000,4.000,4000.000
max,121219.000,6948895.000,4.546,-53.733,650000000.000,821271104.300,46545445.000,200000.000,187000.000,206333.333,3999999.960,3150.000,32.000,10001500.000


In [393]:
df.isnull().sum()

Unnamed: 0                         0
operation                          0
property_type                      0
place_name                        23
place_with_parent_names            0
country_name                       0
state_name                         0
geonames_id                    18717
lat-lon                        51550
lat                            51550
lon                            51550
price                          20410
currency                       20411
price_aprox_local_currency     20410
price_aprox_usd                20410
surface_total_in_m2            39328
surface_covered_in_m2          19907
price_usd_per_m2               52603
price_per_m2                   33562
floor                         113321
rooms                          73830
expenses                      106958
properati_url                      0
description                        2
title                              0
image_thumbnail                 3112
dtype: int64

### Detectamos errores posibles en la carga de la moneda y los corregimos

In [394]:
pricesUSD = df[(
    ~np.isnan(df['price']) & #aca estoy sacando los NaN
    [False if pd.isnull(x) else x=='USD' for x in df['currency']] #aca estoy filtrando los que tengan currency en USD
)].sort_values(by='price_usd_per_m2', ascending=False).head(10);

display('Cantidad de valores en ARS: %d' %len(pricesUSD))
display(pricesUSD['place_name'].unique())

'Cantidad de valores en ARS: 10'

array(['Boedo', 'Ituzaingó', 'Rosario', 'Villa Ballester'], dtype=object)

In [395]:
# reparamos los precios erróneos encontrados
pricesFixed = pricesUSD.copy()
valorDolar = np.round(pricesFixed['price_aprox_local_currency'] / pricesFixed['price_aprox_usd'], 2)

pricesFixed['price_aprox_local_currency'] = pricesFixed['price_aprox_usd']
pricesFixed['price_aprox_usd'] = np.round(pricesFixed['price_aprox_usd'] / valorDolar)
pricesFixed['price'] = pricesFixed['price_aprox_usd']

pricesFixed['price_usd_per_m2'] = pricesFixed['price'] / pricesFixed['surface_covered_in_m2']

pricesFixed.currency = 'ARS'

df.iloc[pricesFixed.index] = pricesFixed

### Duplicados

In [396]:
# Contamos duplicados
df.duplicated(['place_with_parent_names','description','title','floor','rooms','surface_total_in_m2'],keep='first').sum()

9464

In [397]:
# Eliminamos duplicados
df.drop_duplicates(['place_with_parent_names','description','title','floor','rooms','surface_total_in_m2'],keep='first', inplace=True)

### Normalizamos las variables significativas y eliminamos outliers

In [398]:
# Variable: PRICE_USD_PER_M2
# Pasamos los ceros a nan
df["price_usd_per_m2"] = np.where(df["price_usd_per_m2"] == 0, np.nan,df["price_usd_per_m2"])
# Calculamos zscore
#df["price_usd_per_m2_zscore"] = (df["price_usd_per_m2"] - df["price_usd_per_m2"].mean()) / df["price_usd_per_m2"].std()
df["price_usd_per_m2_zscore"] = (df.groupby(['property_type'])[["price_usd_per_m2"]]
.apply(lambda x: (x - x.mean()) / x.std()))
# Contamos outliers
print(df["price_usd_per_m2_zscore"][abs(df["price_usd_per_m2_zscore"]) > 3].count())
# Dropeamos las filas con outliers
df.drop(df["price_usd_per_m2_zscore"][abs(df['price_usd_per_m2_zscore']) > 3].index,inplace=True)
print("")
print(df["price_usd_per_m2"].describe())
# dropeamos la columna temporal zscore
df.drop(columns=["price_usd_per_m2_zscore"], inplace=True)

901

count   62771.000
mean     1881.659
std      1119.417
min         0.600
25%      1166.667
50%      1762.115
75%      2421.053
max     11346.154
Name: price_usd_per_m2, dtype: float64


In [399]:
# Variable: PRICE_USD_PER_M2 - Segunda pasada para limpiar extremos por grupo y zonas
# transformamos en zscore por state_name, place_name y property_type la variable "price_usd_per_m2"
df["price_usd_per_m2_zscore"] = (df.groupby(['state_name','place_name','property_type'])[["price_usd_per_m2"]]
.apply(lambda x: (x - x.mean()) / x.std()))
# y contamos outliers por sobre los 3 desvíos
print(df['price_usd_per_m2_zscore'][abs(df["price_usd_per_m2_zscore"]) > 3].count())
df.drop(df[abs(df["price_usd_per_m2_zscore"])>3].index,inplace=True)
# dropeamos la columna temporal zscore
df.drop(columns=["price_usd_per_m2_zscore"], inplace=True)

630


In [400]:
# Variable: PRICE_APROX_USD
# Pasamos los ceros a nan
df["price_aprox_usd"] = np.where(df["price_aprox_usd"] == 0, np.nan,df["price_aprox_usd"])
# Calculamos Zscore
#df["price_aprox_usd_zscore"] = (df["price_aprox_usd"] - df["price_aprox_usd"].mean()) / df["price_aprox_usd"].std()
df["price_aprox_usd_zscore"] = (df.groupby(['property_type'])[["price_aprox_usd"]]
.apply(lambda x: (x - x.mean()) / x.std()))
# Contamos outliers
print(df["price_aprox_usd_zscore"][abs(df["price_aprox_usd_zscore"]) > 3].count())
# Dropeamos las filas con outliers
df.drop(df["price_aprox_usd"][abs(df['price_aprox_usd_zscore']) > 3].index,inplace=True)
print("")
print(df["price_aprox_usd"].describe())
# dropeamos la columna temporal zscore
df.drop(columns=["price_aprox_usd_zscore"], inplace=True)

1481

count     91429.000
mean     209421.556
std      199658.183
min        4952.450
25%       89733.880
50%      144000.000
75%      255000.000
max     2700000.000
Name: price_aprox_usd, dtype: float64


In [401]:
# Variable: PRICE_APROX_USD - Segunda pasada para limpiar extremos por grupo
# transformamos en zscore por state_name, place_name y property_type la variable "price_usd_per_m2"
df["price_aprox_usd_zscore"] = (df.groupby(['state_name','place_name','property_type'])[["price_aprox_usd"]]
.apply(lambda x: (x - x.mean()) / x.std()))
# y contamos outliers por sobre los 3 desvíos
print(df['price_aprox_usd'][abs(df["price_aprox_usd_zscore"]) > 3].count())
df.drop(df[abs(df["price_aprox_usd_zscore"])>3].index,inplace=True)
# dropeamos la columna temporal zscore
df.drop(columns=["price_aprox_usd_zscore"], inplace=True)

1640


In [402]:
# Variable: SURFACE_TOTAL_IN_M2
# Pasamos los ceros a nan
df["surface_total_in_m2"] = np.where(df["surface_total_in_m2"] == 0, np.nan,df["surface_total_in_m2"])
# Calculamos zscore
#df["surface_total_in_m2_zscore"] = (df["surface_total_in_m2"] - df["surface_total_in_m2"].mean()) / df["surface_total_in_m2"].std()
df["surface_total_in_m2_zscore"] = (df.groupby(['property_type'])[["surface_total_in_m2"]]
.apply(lambda x: (x - x.mean()) / x.std()))
# Contamos outliers
print(df["surface_total_in_m2_zscore"][abs(df["surface_total_in_m2_zscore"]) > 3].count())
# Dropeamos las filas con outliers
df.drop(df["surface_total_in_m2"][abs(df['surface_total_in_m2_zscore']) > 3].index,inplace=True)
print("")
print(df["surface_total_in_m2"].describe())
# dropeamos la columna temporal zscore
df.drop(columns=["surface_total_in_m2_zscore"], inplace=True)
# Como nos vienen muchos valores entre 1 y 25, asumimos es fallido por lo que dropeamos filas < a 25 m2
df.drop(df["surface_total_in_m2"][df["surface_total_in_m2"] < 25].index,inplace=True)

137

count   70327.000
mean      194.109
std       324.235
min        10.000
25%        52.000
50%        87.000
75%       208.000
max      6168.000
Name: surface_total_in_m2, dtype: float64


In [403]:
# Variable: surface_total_in_m2 - Segunda pasada para limpiar extremos por grupo
# transformamos en zscore por state_name, place_name y property_type la variable "price_usd_per_m2"
df["surface_total_in_m2_zscore"] = (df.groupby(['state_name','place_name','property_type'])[["surface_total_in_m2"]]
.apply(lambda x: (x - x.mean()) / x.std()))
# y contamos outliers por sobre los 3 desvíos
print(df['surface_total_in_m2'][abs(df["surface_total_in_m2_zscore"]) > 3].count())
df.drop(df[abs(df["surface_total_in_m2_zscore"])>3].index,inplace=True)
# dropeamos la columna temporal zscore
df.drop(columns=["surface_total_in_m2_zscore"], inplace=True)

1122


In [404]:
# Variable SURFACE_COVERED_IN_M2
# Pasamos los ceros a nan
df["surface_covered_in_m2"] = np.where(df["surface_covered_in_m2"] == 0, np.nan,df["surface_covered_in_m2"])
# Calculamos zscore
#df["surface_covered_in_m2_zscore"] = (df["surface_covered_in_m2"] - df["surface_covered_in_m2"].mean()) / df["surface_covered_in_m2"].std()
df["surface_covered_in_m2_zscore"] = (df.groupby(['property_type'])[["surface_covered_in_m2"]]
.apply(lambda x: (x - x.mean()) / x.std()))
# Contamos outliers
print(df["surface_covered_in_m2_zscore"][abs(df["surface_covered_in_m2_zscore"]) > 3].count())
# Dropeamos las filas con outliers
df.drop(df["surface_covered_in_m2"][abs(df['surface_covered_in_m2_zscore']) > 3].index,inplace=True)
print("")
print(df["surface_covered_in_m2"].describe())
# dropeamos la columna temporal zscore
df.drop(columns=["surface_covered_in_m2_zscore"], inplace=True)
# Como nos vienen muchos valores entre 1 y 25, asumimos es fallido por lo que dropeamos filas < a 25 m2
df.drop(df["surface_covered_in_m2"][df["surface_covered_in_m2"] < 25].index,inplace=True)

75

count   87454.000
mean      117.376
std       117.747
min         1.000
25%        46.000
50%        75.000
75%       150.000
max      2700.000
Name: surface_covered_in_m2, dtype: float64


In [405]:
# Variable: surface_covered_in_m2 - Segunda pasada para limpiar extremos por grupo
# transformamos en zscore por state_name, place_name y property_type la variable "price_usd_per_m2"
df["surface_covered_in_m2_zscore"] = (df.groupby(['state_name','place_name','property_type'])[["surface_covered_in_m2"]]
.apply(lambda x: (x - x.mean()) / x.std()))
# y contamos outliers por sobre los 3 desvíos
print(df['surface_covered_in_m2'][abs(df["surface_covered_in_m2_zscore"]) > 3].count())
df.drop(df[abs(df["surface_covered_in_m2_zscore"])>3].index,inplace=True)
# dropeamos la columna temporal zscore
df.drop(columns=["surface_covered_in_m2_zscore"], inplace=True)

1038


In [406]:
# Variable rooms
# Pasamos los ceros a nan
df["rooms"] = np.where(df["rooms"] == 0, np.nan,df["rooms"])
# Calculamos zscore
#df["rooms_zscore"] = (df["rooms"] - df["rooms"].mean()) / df["rooms"].std()
df["rooms_zscore"] = (df.groupby(['property_type'])[["rooms"]]
.apply(lambda x: (x - x.mean()) / x.std()))
# Contamos outliers
print(df["rooms_zscore"][abs(df["rooms_zscore"]) > 3].count())
# Dropeamos las filas con outliers
df.drop(df["rooms"][abs(df['rooms_zscore']) > 3].index,inplace=True)
# dropeamos la columna temporal zscore
df.drop(columns=["rooms_zscore"], inplace=True)
# Como nos vienen muchos valores altos, asumimos es fallido por lo que dropeamos filas > a 11
df.drop(df["rooms"][df["rooms"] > 10].index,inplace=True)
print("")
print(df["rooms"].describe())

382

count   39630.000
mean        2.972
std         1.498
min         1.000
25%         2.000
50%         3.000
75%         4.000
max        10.000
Name: rooms, dtype: float64


In [407]:
# Variable expenses
# Pasamos los ceros a nan
df["expenses"] = np.where(df["expenses"] == 0, np.nan,df["expenses"])
# Calculamos zscore
#df["expenses_zscore"] = (df["expenses"] - df["expenses"].mean()) / df["expenses"].std()
df["expenses_zscore"] = (df.groupby(['state_name','property_type'])[["expenses"]]
.apply(lambda x: (x - x.mean()) / x.std()))
# Contamos outliers
print(df["expenses_zscore"][abs(df["expenses_zscore"]) > 3].count())
# Dropeamos las filas con outliers
df.drop(df["expenses"][abs(df['expenses_zscore']) > 3].index,inplace=True)
print("")
print(df["expenses"].describe())
# dropeamos la columna temporal zscore
df.drop(columns=["expenses_zscore"], inplace=True)

97

count    12511.000
mean      2778.704
std       5824.894
min          1.000
25%       1000.000
50%       1900.000
75%       3700.000
max     380000.000
Name: expenses, dtype: float64


In [408]:
# Variable floor
# Pasamos los ceros a nan
df["floor"] = np.where(df["floor"] == 0, np.nan,df["floor"])
# Calculamos zscore
df["floor_zscore"] = (df["floor"] - df["floor"].mean()) / df["floor"].std()
#df["floor_zscore"] = (df.groupby(['state_name'])[["floor"]]
#.apply(lambda x: (x - x.mean()) / x.std()))
# Contamos outliers
print(df["floor_zscore"][abs(df["floor_zscore"]) > 3].count())
# Dropeamos las filas con outliers
df.drop(df["floor"][abs(df['floor_zscore']) > 3].index,inplace=True)
# dropeamos la columna temporal zscore
df.drop(columns=["floor_zscore"], inplace=True)
# Como nos vienen muchos valores altos, asumimos es fallido por lo que dropeamos filas > a 55 (edif + alto Arg)
df.drop(df['floor'][df["floor"] > 55].index,inplace=True)
print("")
print(df["floor"].describe())

83

count   6644.000
mean       4.317
std        4.278
min        1.000
25%        2.000
50%        3.000
75%        6.000
max       54.000
Name: floor, dtype: float64


### Resolvemos los faltantes de "place_name" (25 casos)

In [409]:
# Rellenamos los valores faltantes en "place_name" parseando el campo "place_with_parent_names"
df["place_name"].fillna(df['place_with_parent_names'].str.extract('.*\|(.*)\|.*', expand=True)[0],inplace=True)

### Descartamos que las otras columnas de precio nos sirvan para imputar superficies donde falten, y que existan datos de precio cuando price_aprox_usd sea nula

In [410]:
df['surface_total_in_m2'][(df['surface_total_in_m2'].isnull() & df['price_aprox_usd'].notna() & df['price_usd_per_m2'].notna())].count()

0

In [411]:
# Hay datos de precio x m2 en filas que no tengan datos de superficie?
df["price_usd_per_m2"].loc[(df["surface_total_in_m2"].isnull() & df["surface_covered_in_m2"].isnull())].count()

0

In [412]:
df["price_per_m2"].loc[(df["surface_total_in_m2"].isnull() & df["surface_covered_in_m2"].isnull())].count()

0

In [413]:
# Hay datos de precio en moneda original para los campos en null en "price_aprox_usd"?
df["price"].loc[df["price_aprox_usd"].isnull()].count()
#print(df["price_aprox_local_currency"].loc[df["price_aprox_usd"].isnull()].sum())

1

### Limpiamos y rellenamos variable de Superficie - Imputamos "surface_covered_in_m2" a "surface_total_in_m2" donde la segunda es nula pero hay datos en la primera

In [414]:
# la media de superficie antes de imputar
df["surface_total_in_m2"].mean()

180.5609178736814

In [415]:
# rellenamos
df["surface_total_in_m2"] = \
np.where(df["surface_total_in_m2"].isnull(),df["surface_covered_in_m2"],df["surface_total_in_m2"])
# limpiamos valores en cero y los pasamos a nan
df["surface_total_in_m2"] = np.where(df["surface_total_in_m2"] == 0, np.nan,df["surface_total_in_m2"])

In [416]:
# la media de superficie despues de imputar
df["surface_total_in_m2"].mean()

166.45709861414375

In [417]:
# chequeamos que la imputación haya sido completa
df["surface_covered_in_m2"].loc[df["surface_total_in_m2"].isnull()].count()

0

In [418]:
# Conteo cuantos m2 en nulos
df["surface_total_in_m2"].isnull().sum()

11747

In [419]:
# Controlamos si en los casos donde no existe el dato de superficie si existen datos de precio
print(df["price_per_m2"][df["surface_total_in_m2"].isnull()].sum())
print(df["price_usd_per_m2"][df["surface_total_in_m2"].isnull()].sum())
print(df["price_aprox_usd"][df["surface_total_in_m2"].isnull()].count())
print(df["price_aprox_local_currency"][df["surface_total_in_m2"].isnull()].count())
print(df["price"][df["surface_total_in_m2"].isnull()].count())

0.0
0.0
7229
7230
7230


In [420]:
# dropeamos todas las filas que no tienen datos de superficie - 12mil filas
df = df[df["surface_total_in_m2"].notna()]

### Parseamos el campo "description" para rescatar datos de ambientes

In [421]:
# en busca de cantidad de dormitorios
df["dorm_en_desc"] = df['description'].str.extract('(\d+?) (dormitorios|dormitorio)')[0]
# contamos cuantos datos ganamos
print(df["dorm_en_desc"].loc[df["rooms"].isnull()].count())
# imputamos sumando 1 para transformar dormitorios en ambientes
df["rooms"] = np.where(df["rooms"].isnull(),pd.to_numeric(df["dorm_en_desc"]) + 1,df["rooms"])
# dropeamos la columna intermedia
df.drop(columns=["dorm_en_desc"], inplace=True)

13436


In [422]:
# en busca de cantidad de ambientes
df["amb_en_desc"] = df['description'].str.extract('(\d+?) (ambientes|ambiente)')[0]
# contamos cuantos datos ganamos
print(df["amb_en_desc"].loc[df["rooms"].isnull()].count())
# imputamos suma
df["rooms"] = np.where(df["rooms"].isnull(),pd.to_numeric(df["amb_en_desc"]),df["rooms"])
# dropeamos la columna intermedia
df.drop(columns=["amb_en_desc"], inplace=True)

6101


### Parseamos el campo "title" para rescatar datos de superficie

In [423]:
# en busca de cantidad de dormitorios
df["dorm_en_tit"] = df['title'].str.extract('(\d+?) (dormitorios|dormitorio)')[0]
# y contamos cuantos datos ganamos
print(df["dorm_en_tit"].loc[df["rooms"].isnull()].count())
# imputamos sumando 1 para transformar dormitorios en ambientes
df["rooms"] = np.where(df["rooms"].isnull(),pd.to_numeric(df["dorm_en_tit"]) + 1,df["rooms"])
# dropeamos la columna intermedia
df.drop(columns=["dorm_en_tit"], inplace=True)

446


In [424]:
# en busca de cantidad de ambientes
df["amb_en_tit"] = df['title'].str.extract('(\d+?) (ambientes|ambiente)')[0]
# contamos cuantos datos ganamos
print(df["amb_en_tit"].loc[df["rooms"].isnull()].count())
# imputamos
df["rooms"] = np.where(df["rooms"].isnull(),pd.to_numeric(df["amb_en_tit"]),df["rooms"])
# dropeamos la columna intermedia
df.drop(columns=["amb_en_tit"], inplace=True)

776


In [425]:
# Volvemos a limpiar el máximo de 10 ambientes para la variable rooms
df.drop(df["rooms"][df["rooms"] > 10].index,inplace=True)

### Rellenamos faltantes la variable price usd per m2 cuando hayan datos de precio total y superficie

In [426]:
# recalcular el precio usd por m2 donde este sea null
print(df["price_usd_per_m2"].isnull().sum()) # nulos antes
df["price_usd_per_m2"] = np.where(df["price_usd_per_m2"].isnull(),np.divide(df["price_aprox_usd"],df["surface_total_in_m2"]),df["price_usd_per_m2"])
print(df["price_usd_per_m2"].isnull().sum()) # nulos despues

31878
11797


### Resolvemos los valores faltantes del campo "price_usd_per_m2"

In [427]:
# imputar a los valores faltantes de price_usd_per_m2 con el promedio por provincia, localidad/barrio y tipo de propiedad
df["price_usd_per_m2"].fillna(df.groupby(['state_name','property_type'])['price_usd_per_m2'].transform('median'),inplace=True)
#df.groupby(['state_name','place_name','property_type'])[['price_usd_per_m2']].transform('median').sort_values('price_usd_per_m2',ascending=True)
#df.sort_values('price_usd_per_m2',ascending=False)
# dropear remanente que no pudo ser rellenado con promedios
df.drop(df[df["price_usd_per_m2"].isnull()].index,inplace=True)

In [428]:
# rellenar precios totales en USD faltantes
df['price_aprox_usd'] = np.where(df['price_aprox_usd'].isnull(),df['price_usd_per_m2'] * df['surface_total_in_m2'],df['price_aprox_usd'])

### Creación nueva variable Zona recategorizando state_name

In [429]:
df['state_name'].unique()

array(['Capital Federal', 'Buenos Aires Costa Atlántica', 'Entre Ríos',
       'Bs.As. G.B.A. Zona Norte', 'Córdoba', 'Bs.As. G.B.A. Zona Oeste',
       'Bs.As. G.B.A. Zona Sur', 'Misiones', 'Santa Fe',
       'Buenos Aires Interior', 'Neuquén', 'Río Negro', 'San Luis',
       'Mendoza', 'Corrientes', 'Chubut', 'Tucumán', 'Chaco', 'Salta',
       'San Juan', 'Tierra Del Fuego', 'Santa Cruz', 'La Pampa',
       'Santiago Del Estero', 'Catamarca', 'Jujuy', 'La Rioja', 'Formosa'],
      dtype=object)

In [430]:
# Reemplazos valores y creamos nueva variable
df['zona'] = df['state_name'].replace({'Capital Federal':'Capital & GBA', 'Buenos Aires Costa Atlántica':'Interior', 'Entre Ríos':'Interior', 'Bs.As. G.B.A. Zona Norte':'Capital & GBA', 'Córdoba':'Interior', 'Bs.As. G.B.A. Zona Oeste':'Capital & GBA', 'Bs.As. G.B.A. Zona Sur':'Capital & GBA', 'Misiones':'Interior', 'Santa Fe':'Interior', 'Buenos Aires Interior':'Interior', 'Neuquén':'Interior', 'Río Negro':'Interior', 'San Luis':'Interior', 'Mendoza':'Interior', 'Corrientes':'Interior', 'Chubut':'Interior', 'Tucumán':'Interior', 'Chaco':'Interior', 'Salta':'Interior', 'San Juan':'Interior', 'Tierra Del Fuego':'Interior', 'Santa Cruz':'Interior', 'La Pampa':'Interior', 'Santiago Del Estero':'Interior', 'Catamarca':'Interior', 'Jujuy':'Interior', 'La Rioja':'Interior', 'Formosa':'Interior'})
df['zona'].value_counts()

Capital & GBA    63132
Interior         26589
Name: zona, dtype: int64

### Limpieza final y evaluación del dataset resultante

In [431]:
# Nos quedamos solo con los precios en USD
df.drop(columns=["price","price_aprox_local_currency","price_per_m2","currency"], inplace=True)

In [432]:
# Limpiamos el Dataset de Columnas innecesarias
df.drop(columns=["Unnamed: 0","operation","place_with_parent_names","country_name","lat-lon","image_thumbnail"], inplace=True)

In [433]:
# Mapeamos los valores válidos con los que nos quedamos
df.notna().sum()

property_type            89721
place_name               89721
state_name               89721
geonames_id              74770
lat                      51889
lon                      51889
price_aprox_usd          89721
surface_total_in_m2      89721
surface_covered_in_m2    83887
price_usd_per_m2         89721
floor                     6216
rooms                    56308
expenses                 12069
properati_url            89721
description              89720
title                    89721
zona                     89721
dtype: int64

In [434]:
df.describe()

,geonames_id,lat,lon,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,floor,rooms,expenses
count,74770.000,51889.000,51889.000,89721.000,89721.000,83887.000,89721.000,6216.000,56308.000,12069.000
mean,3561684.280,-34.730,-59.076,208761.667,166.431,116.192,1773.945,4.277,3.013,2802.150
std,367957.304,1.841,2.053,202348.918,242.566,110.208,932.112,4.227,1.308,5912.947
min,3427208.000,-54.820,-71.665,1323.332,25.000,25.000,8.062,1.000,1.000,1.000
25%,3430234.000,-34.666,-58.690,92000.000,52.000,47.000,1156.069,2.000,2.000,1000.000
50%,3433775.000,-34.599,-58.473,145000.000,85.000,76.000,1666.667,3.000,3.000,1900.000
75%,3436397.000,-34.473,-58.395,250000.000,190.000,150.000,2300.000,6.000,4.000,3800.000
max,6948895.000,-23.137,-54.556,7664487.633,6168.000,2700.000,16666.667,54.000,10.000,380000.000


In [435]:
df.groupby(['state_name'])[['price_usd_per_m2']].mean().sort_values(by=['price_usd_per_m2'],ascending=False).head(10)

,price_usd_per_m2
state_name,
Capital Federal,2483.071
Catamarca,1890.291
Bs.As. G.B.A. Zona Norte,1775.907
Río Negro,1637.077
San Juan,1566.091
Neuquén,1549.860
Buenos Aires Costa Atlántica,1545.373
Chubut,1531.872
Formosa,1512.175


In [436]:
df.groupby(['state_name','place_name','property_type'])[['price_usd_per_m2']].mean().sort_values(by=['price_usd_per_m2'],ascending=False).head(15)

,,,price_usd_per_m2
state_name,place_name,property_type,
Buenos Aires Interior,Pergamino,store,10000.000
Córdoba,Cosquín,store,7500.000
Capital Federal,Puerto Madero,store,5517.406
La Pampa,Perú,apartment,5200.000
Córdoba,Oliva,house,5196.633
Capital Federal,Puerto Madero,apartment,5194.173
Bs.As. G.B.A. Zona Norte,Barrio Los Alisos,store,5050.505
Catamarca,Manantiales,apartment,4964.115
Mendoza,Capilla del Rosario,house,4375.000


In [437]:
df.to_pickle('dataset.pkl')